# Session 11 - Generative language models for zero-shot learning

In [1]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We're going to be working with ```FLAN-T5```, a text-to-text model developed by Google. ```FLAN-T5``` is based on ```T5```, which we saw in the lecture, but it has been further finetuned on a range of common text-to-text tasks. This means that it can already perform a lot of the kinds of tasks that people use generative language models for. You can read more in the paper here: [https://arxiv.org/pdf/2210.11416.pdf](https://arxiv.org/pdf/2210.11416.pdf)

We load the model from ```huggingface```. We're here using the ```Large``` version, but you can try the other sizes if you want. The ```Large``` version is already 3.4GB, and even larger models will take a long time to download and to run - but they should also see a marked performance improvement.
 
You can read more about the available models here: [https://huggingface.co/docs/transformers/model_doc/flan-t5](https://huggingface.co/docs/transformers/model_doc/flan-t5)

In [4]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tf_keras\src\initializers\initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


We also download and initalize the pretrained tokenizer that fits with our model:

In [13]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

## Prompt engineering

Our goal is to use the knowledge of language that FLAN-T5 has already acquired during training and to use that knowledge in different domains *without any further fine-tuning*. This is an example of what is called *zero-shot* learning.

In order for zero-shot learning to be successful, our prompts need to be carefully designed. FLAN-T5 (and similar models) are a bit less flexible than, for example, ChatGPT.

In [6]:
# classification
prompt = "classify the following text as positive or negative: I absolutely hated this movie"

# translation
#prompt = "translate from English to French: how old are you?"

# question answering
#prompt = "answer the following question: how is cheese made?"

# named entity recognition
#prompt = "find all location entities in this text: Ross comes from Scotland"

We then pass our text prompt to the tokenizer, defing some extra arguments such as the ```max_length``` of our input (anything longer than this will be truncated):

In [7]:
inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


We then pass all of our input prompt tokens to the model and use than to generate an appropriate output from what FLAN-T5 has learned during training.

In [8]:
outputs = model.generate(**inputs)

c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [9]:
print(tokenizer.batch_decode(outputs, 
                            skip_special_tokens=True))

['negative']


We can also do this a bit more cleverly by using a single prompt plus a F-string:

In [ ]:
prompt = f"classify the following text as positive or negative: {input_text}"

This means we could, for example, write functions for specific tasks:

In [11]:
def classifier(input_text:str) -> str:
    prompt = f"classify the following text as positive or negative: {input_text}"
    inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [11]:
classifier("I absolutely hated this movie!")

['negative']


## Tasks

Look through previous notebooks, exercises, and datasets from Language Analytics so far this semester. In small groups, try using either ```Flan-T5``` and **ChatGPT** (or both) try to solve those problems using generative language models.

So that would mean, for example:

    - Grammatical analysis
    - Named entity recognition/extraction
    - Classification
    - Topic modelling

As an illustrative example: try using Flan-T5 to perform classification on the Fake or Real News dataset. How does it perform on ground truth? Is it better or worse than the other models we've seen? Be creative!


In [3]:
import pandas as pd

In [4]:
pd.read_csv("ArticlesApril2017.csv")

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,NaN,58fd41ab7c459f24986dbaa7,710,By ANDREW E. KRAMER,article,Reporting on Gays Who ‘Don’t Exist’,"['Chechnya (Russia)', 'Homosexuality and Bisex...",3,Insider,2,2017-04-24 00:07:04,Unknown,"“I see flies, I see mosquitoes,” said a Cheche...",The New York Times,News,https://www.nytimes.com/2017/04/23/insider/rus...
882,NaN,58fd45a17c459f24986dbaaa,1230,By MATT FLEGENHEIMER and THOMAS KAPLAN,article,The Fights That Could Lead to a Government Shu...,"['Trump, Donald J', 'United States Politics an...",3,National,15,2017-04-24 00:23:53,Politics,The Trump administration wants to use the dead...,The New York Times,News,https://www.nytimes.com/2017/04/23/us/politics...
883,NaN,58fd5c2c7c459f24986dbac3,1424,By NOEL MURRAY,article,"‘The Leftovers’ Season 3, Episode 2: Swedish P...","['Television', 'The Leftovers (TV Program)']",3,Culture,0,2017-04-24 02:00:04,Television,"For all its melancholy, “The Leftovers” rarely...",The New York Times,Review,https://www.nytimes.com/2017/04/23/arts/televi...
884,NaN,58fd5c3d7c459f24986dbac4,1052,By BEN BRANTLEY,article,"Thinking Out Loud, But Why?","['Theater', 'The Antipodes (Play)', 'Baker, An...",3,Culture,1,2017-04-24 02:00:25,Unknown,"In this endlessly fascinating work, Annie Bake...",The New York Times,Review,https://www.nytimes.com/2017/04/23/theater/the...


In [8]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("fake_or_real_news.csv")

# Access the headline column
headline_column = df['text']

# Print the headline column
print(headline_column[0])


Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. 
In the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. 
The word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. 
But that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. 
The FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. 
The FBI’s leadership is being warned that the entire left

In [17]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tf_keras\src\initializers\initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [20]:
def classifier(input_text:str) -> str:
    prompt = f"classify the following news article as real or fake: {input_text}"
    inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [30]:
y_label = df['label'][0]
print(y_label)

FAKE


In [31]:
y_pred = classifier(df['text'][0])

c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['real']
